# Lab 3.2 — Pipeline giữ làn hoàn chỉnh (Perception → Geometry → Control → Overlay)

Lab này tích hợp toàn bộ các thành phần đã được xây dựng trong các lab trước:

#### Từ Lab 1 (Perception)
- Trích xuất mặt nạ làn bằng deep learning (YOLO / PIDNet / TwinLite / BiSeNet)

#### Từ Lab 2 (Geometry)
- Lọc ROI
- Tinh chỉnh hình thái học
- Chiếu Bird’s-Eye View (BEV)

#### Từ Lab 3.1 (Lane Geometry)
- center_x() để phát hiện tâm làn
- heading_deg_at_ratio() từ hai điểm lấy mẫu theo phương dọc
- meters_per_pixel() để chuyển px → mét
- Lấy mẫu đa tỉ lệ (r = 0.98, 0.92, 0.82, 0.72)
- Chọn tỉ lệ thích ứng dựa trên độ ổn định

### Trong Lab này bạn sẽ:
1. Triển khai bộ điều khiển đánh lái đơn giản.
2. Chuyển hình học làn → LEFT / RIGHT / STRAIGHT.
3. Vẽ overlay thời gian thực lên các khung hình video.
4. Chạy toàn bộ pipeline giữ làn trên một video đầu vào thực tế.


### Environment Setup

Lab này yêu cầu nhiều thư viện xử lý ảnh và tính toán số.
Chúng ta sẽ cài đặt và kiểm tra tất cả các dependencies trước khi chạy pipeline
hình học và điều khiển.

**Required Libraries**
- **numpy** — thao tác ma trận  
- **opencv-python** — xử lý ảnh/video  
- **matplotlib** — trực quan hóa  
- **torch** — tải các mô hình lane deep learning  
- **ultralytics** (tùy chọn) — các mô hình YOLO cho lane/segmentation  

Hãy đảm bảo môi trường runtime có GPU nếu bạn muốn chạy backend deep learning.
Chế độ CPU-only vẫn chấp nhận được cho lab này.


In [ ]:
!pip install numpy opencv-python matplotlib ultralytics --quiet
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet

print("All dependencies installed successfully.")

All dependencies installed successfully.


### Import Libraries

Chúng ta import tất cả các thư viện tiêu chuẩn được sử dụng trong suốt lab:

- numpy cho tính toán số  
- cv2 cho xử lý ảnh  
- matplotlib cho trực quan hóa  
- Tùy chọn: torch + ultralytics cho backend segmentation  


In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO

### Version Check

Cần xác nhận rằng:
- OpenCV đã được cài đặt đúng  
- Torch nhận diện được CUDA (tùy chọn)  


In [ ]:
print("OpenCV version:", cv2.__version__)
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

### 1. Perception Module: YOLOv8 Backend

> **Note:** Module này đã được triển khai đầy đủ trong **Lab 1**. Trong Lab 3, chúng ta xem nó như một **cảm biến "Black Box"** cung cấp mặt nạ làn đường nhị phân.

Chúng ta sử dụng class `YoloV8Backend` để:
1.  Tải trọng số đã huấn luyện (`best.pt`).
2.  Thực hiện suy luận trên từng khung hình video.
3.  Trả về mặt nạ phân đoạn nhị phân (`0` cho nền, `1` cho làn đường).

**Action:** Chỉ cần **chạy cell bên dưới** để khởi tạo backend. Không cần viết code tại đây.


In [ ]:
class YoloV8Backend:
    """
    Lightweight wrapper for YOLOv8 segmentation → binary lane mask.
    Matches real project structure but simplified for lab environment.
    """
    def __init__(self, weights, device="cuda", imgsz=640, conf=0.18):
        self.model = YOLO(weights)
        self.device = device
        self.imgsz = imgsz
        self.conf = conf
        self.fp16 = (torch.cuda.is_available() and device != "cpu")
        self.model.to(device)
        self.model.fuse()

    def infer_mask01(self, frame_bgr):
        H, W = frame_bgr.shape[:2]
        res = self.model.predict(frame_bgr, imgsz=self.imgsz, conf=self.conf,
                                 device=self.device, verbose=False, half=self.fp16)
        r0 = res[0]
        if r0.masks is None:
            return np.zeros((H, W), np.uint8)

        mk = r0.masks.data.cpu().numpy().astype(np.uint8)
        merged = np.zeros((H, W), np.uint8)
        for k in range(mk.shape[0]):
            merged = cv2.bitwise_or(merged, cv2.resize(mk[k], (W, H)))

        return merged


In [ ]:
backend = YoloV8Backend(r"C:\Users\admin\ACE_Finalv4\AI\LaneDetection\Lane_weight\Yolo_v8\best.pt")
print("Backend loaded:", backend.name() if hasattr(backend, "name") else "YOLOv8")

### 2. Preprocessing Module: ROI & BEV (Recap from Lab 2)

**Context:** Trong **Lab 2**, chúng ta đã xây dựng một pipeline tiền xử lý mạnh mẽ để làm sạch mặt nạ phân đoạn và biến đổi nó thành Bird's-Eye-View (BEV). Chúng ta sẽ tái sử dụng các tiện ích này tại đây.

Module này bao gồm:
1. **apply_roi**: Cắt vùng quan tâm (loại bỏ bầu trời/phần nền).
2. **refine_mask01**: Áp dụng các phép toán hình thái học (Opening/Closing) để giảm nhiễu và lấp đầy khe hở.
3. **BEVProjector**: Một class quản lý ma trận Homography để warp ảnh từ *Perspective View* sang *Top-Down View*.

**Action:** Chạy cell bên dưới để định nghĩa các hàm hỗ trợ này.


In [ ]:
def apply_roi(mask, poly):
    H, W = mask.shape
    pts = np.array([(int(x*W), int(y*H)) for x,y in poly], dtype=np.int32)
    roi = np.zeros_like(mask)
    cv2.fillPoly(roi, [pts], 1)
    return mask * roi

def refine_mask01(mask):
    k = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)
    return mask

class BEVProjector:
    def __init__(self):
        self.src = ((0.20, 0.58),(0.10, 0.90),(0.90, 0.90),(0.80, 0.58))
        self.dst = ((0.25, 0.00),(0.25, 1.00),(0.75, 1.00),(0.75, 0.00))
        self.M = None
        self.M_inv = None
        self._wh = None

    def _ensure(self, W, H):
        if self._wh == (W,H): return
        src = np.float32([(x*W,y*H) for x,y in self.src])
        dst = np.float32([(x*W,y*H) for x,y in self.dst])
        self.M = cv2.getPerspectiveTransform(src, dst)
        self.M_inv = cv2.getPerspectiveTransform(dst, src)
        self._wh = (W,H)

    def warp(self, mask):
        H,W = mask.shape
        self._ensure(W,H)
        bev = cv2.warpPerspective(mask*255, self.M, (W,H), cv2.INTER_NEAREST)
        return (bev>0).astype(np.uint8)


### Geometry Math ( Recap from Lab 3.1)

Trong phần này, chúng ta định nghĩa lại toàn bộ các hàm lane-geometry cần thiết cho pipeline hoàn chỉnh.  
Mặc dù các hàm này đã được triển khai trong Lab 3.1, Jupyter notebooks yêu cầu phải khai báo lại chúng trong Lab 3.2.

Khối này cung cấp:
- center_x() — trích tâm làn tại một tỉ lệ BEV cho trước  
- heading_deg_at_ratio() — tính góc hướng bằng hai điểm lấy mẫu theo chiều dọc  
- multi_ratio_measure() — đánh giá nhiều tỉ lệ (0.98, 0.92, 0.82, 0.72)  
- adaptive_select_solution() — chọn tỉ lệ có ước lượng hình học ổn định nhất  

Các hàm này chuyển BEV mask đã tinh chỉnh thành các đại lượng hình học thực-valued được sử dụng bởi controller.


In [ ]:
def center_x(bev, r):
    H, W = bev.shape
    y = int(r * H)
    xs = np.where(bev[y] > 0)[0]
    if len(xs) < 2:
        return np.nan
    return 0.5 * (xs[0] + xs[-1])

def heading_deg_at_ratio(bev, r, dy=30):
    H, W = bev.shape
    y = int(r * H)
    y2 = max(0, y - dy)

    xs1 = np.where(bev[y] > 0)[0]
    xs2 = np.where(bev[y2] > 0)[0]

    if len(xs1)<2 or len(xs2)<2:
        return np.nan, np.nan

    cx1 = 0.5*(xs1[0]+xs1[-1])
    cx2 = 0.5*(xs2[0]+xs2[-1])
    dx = cx1 - cx2
    dy = (y - y2)

    ang = np.degrees(np.arctan2(dx, dy + 1e-6))
    return ang, cx1

def multi_ratio_measure(bev, ratios, dy_px=30, lane_width_m=0.20):
    H, W = bev.shape
    mpp = lane_width_m / 20.0  

    centers, pos_list, head_list = [], [], []
    for r in ratios:
        head, cx = heading_deg_at_ratio(bev, r, dy_px)
        pos_m = (W/2 - cx) * mpp
        centers.append(cx)
        pos_list.append(pos_m)
        head_list.append(head)

    return centers, pos_list, head_list


In [ ]:
def adaptive_select(centers, pos_list, head_list, ratios,
                             W_STAB=2.0, W_HEAD=0.1, W_LAT=0.05):
    best_i = 0
    best_score = -1e9

    for i,r in enumerate(ratios):
        cx = centers[i]
        if np.isnan(cx): continue

        std_pos = abs(pos_list[i])
        mean_head = abs(head_list[i])
        score = (W_STAB/(std_pos+1e-6) - W_HEAD*mean_head - W_LAT*(1-r))

        if score > best_score:
            best_score = score
            best_i = i

    return {
        "best_ratio": ratios[best_i],
        "pos": pos_list[best_i],
        "head": head_list[best_i]
    }


### Controller

Controller chuyển hình học làn đường thành lệnh đánh lái để giữ xe ở giữa làn.  
Nó kết hợp hai loại thông tin hình học:

1. **Lateral offset** (pos_m):  
   Độ lệch ngang của xe so với tâm làn (tính theo mét).  
   - Nếu pos_m > 0 → xe lệch sang PHẢI → phải đánh lái SANG TRÁI  
   - Nếu pos_m < 0 → xe lệch sang TRÁI → phải đánh lái SANG PHẢI  

2. **Heading angle** (head_deg):  
   Góc hướng của làn so với hướng tiến của xe.  
   - Góc dương → làn cong sang trái  
   - Góc âm → làn cong sang phải  

Để hiệu chỉnh quỹ đạo xe, ta dùng một công thức điều khiển tuyến tính kết hợp hai tín hiệu này:

#### **Control law**

$$
\text{steer} = k_{\text{pos}} \cdot \text{pos}_m \;+\; k_{\text{head}} \cdot \text{head}_{deg}
$$

#### Ý nghĩa từng thành phần:

- $( k_{\text{pos}} \cdot \text{pos}_m $):  
  Sửa sai lệch vị trí so với tâm làn → **đóng góp chính vào đánh lái**.

- $( k_{\text{head}} \cdot \text{head}_{deg} $):  
  Dự đoán độ cong của làn để hiệu chỉnh sớm hơn → **ổn định khi vào cua**.

Các hệ số:
- $( k_{\text{pos}} $): mức độ controller phản ứng với độ lệch ngang  
- $( k_{\text{head}} $): mức độ phản ứng với độ cong làn  

Các giá trị này quyết định độ “gắt” hay “mượt” của đánh lái.

Đầu ra đánh lái cuối cùng bị ràng buộc trong giới hạn phần cứng:

$$
\text{steer} \in [-50, 50]
$$

#### Nhãn đánh lái (phục vụ trực quan hóa)

Để dễ diễn giải giá trị đánh lái, ta chuyển nó thành ba nhãn:

- **LEFT** → steer > threshold  
- **RIGHT** → steer < −threshold  
- **STRAIGHT** → giá trị nhỏ, xem như không có đổi hướng đáng kể  

Phân loại này chỉ phục vụ hiển thị, không ảnh hưởng đến điều khiển.


In [ ]:
def controller(pos_m, head_deg, k_pos=40.0, k_head=1.5):
    steer = k_pos * pos_m + k_head * head_deg
    steer = np.clip(steer, -50, 50)
    return float(steer)

def steering_label(steer, thresh=3.0):
    if steer > thresh:
        return "LEFT"
    elif steer < -thresh:
        return "RIGHT"
    else:
        return "STRAIGHT"
    

### Draw Overlay

Hàm này trực quan hóa toàn bộ đầu ra hình học của làn đường lên trên khung hình camera gốc.  
Nó cung cấp cách nhìn trực quan giúp hiểu pipeline đang xử lý gì ở từng bước.

Overlay bao gồm:

#### **1. BEV Mask Projected Back to Camera View**
Mặt nạ BEV nhị phân (top-down view) được inverse-warp bằng homography nghịch $( M^{-1} $)  
để vùng làn đường có thể chạy được hiển thị lại trên ảnh camera.  
Điều này giúp kiểm tra xem phép biến đổi BEV và tinh chỉnh mask có chính xác không.

#### **2. Multi-Ratio Lane Centers**
Với mỗi tỉ lệ lấy mẫu (ví dụ 0.98 → 0.92 → 0.82 → 0.72):

- Điểm tâm làn được phát hiện trong tọa độ BEV được ánh xạ ngược sang tọa độ camera.
- Tất cả các điểm tỉ lệ được vẽ:
  - **Primary ratio** (tỉ lệ có điểm ổn định cao nhất) → chấm ĐỎ  
  - **Các tỉ lệ hợp lệ khác** → chấm VÀNG  

Điều này giúp quan sát:
- liệu phát hiện tâm làn có ổn định giữa các tỉ lệ không  
- liệu cơ chế adaptive ratio selection có hoạt động đúng không  

#### **3. On-Screen Debug Information**
Một bảng thông tin nhỏ được vẽ ở góc trên bên trái hiển thị:

- Lateral offset theo mét  
- Heading angle theo độ  
- Nhãn điều khiển đánh lái (“LEFT”, “RIGHT”, “STRAIGHT”)  

Các giá trị này giúp liên hệ overlay trực quan với đầu ra hình học và controller.

#### **Purpose**
Bước trực quan hóa này rất quan trọng cho việc debug:
- lỗi segmentation  
- sai lệch BEV  
- bất ổn hình học  
- lỗi chọn tỉ lệ  
- hành vi controller  

Hàm trả về một frame đã được annotate để hiển thị hoặc lưu.


In [ ]:
def draw_overlay(frame_bgr, bev01, M_inv, ratios, centers_px, primary_idx, pos_m, head_deg, dir_label):
    draw = frame_bgr.copy()
    H, W = draw.shape[:2]
    # ============================
    # (1) Warp BEV mask về camera
    # ============================
    if bev01 is not None and M_inv is not None:
        bev_warp = cv2.warpPerspective(
            (bev01 * 255).astype(np.uint8),
            M_inv, (W, H),
            flags=cv2.INTER_NEAREST
        )
        green = np.zeros_like(draw)
        green[bev_warp > 0] = (0, 180, 0)
        draw = cv2.addWeighted(draw, 1.0, green, 0.8, 0)

    # ============================
    # (2) Chuyển điểm BEV → camera
    # ============================
    pts_bev = []
    for r, cx in zip(ratios, centers_px):
        if np.isnan(cx):
            pts_bev.append([np.nan, np.nan])
        else:
            y_bev = r * bev01.shape[0]
            pts_bev.append([cx, y_bev])

    valid_pts = np.array(
        [[px, py] for px, py in pts_bev if not np.isnan(px)],
        dtype=np.float32
    ).reshape(-1, 1, 2)

    if valid_pts.shape[0] > 0:
        pts_cam = cv2.perspectiveTransform(valid_pts, M_inv)
    else:
        pts_cam = []

    # ============================
    # (3) Vẽ các điểm ratio
    # ============================
    cam_i = 0
    for i, (r, cx) in enumerate(zip(ratios, centers_px)):
        if np.isnan(cx):
            continue
        px, py = pts_cam[cam_i][0]
        cam_i += 1
        if i == primary_idx:
            cv2.circle(draw, (int(px), int(py)), 5, (0,0,255), -1)   # đỏ
        else:
            cv2.circle(draw, (int(px), int(py)), 3, (0,255,255), -1) # vàng

    # ============================
    # (4) Ô thông góc, hướng
    # ============================

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(draw, f"Pos: {pos_m:+.3f} m", (15,20), font, 0.45, (0,255,255), 2)
    cv2.putText(draw, f"Head: {head_deg:+.2f} deg", (15,40), font, 0.55, (0,255,255), 2)
    color_dir = (0,255,0) if dir_label=="STRAIGHT" else (0,0,255)
    cv2.putText(draw, f"DIR: {dir_label}", (15,60), font, 0.65, color_dir, 2)
    return draw


### Full Pipeline Execution (Video Processing)

Trong nhiệm vụ cuối cùng này, bạn sẽ chạy **toàn bộ pipeline giữ làn** trên một video lái xe đã ghi sẵn.
Nội dung này tích hợp toàn bộ các module đã phát triển trong các lab trước:

- **Perception (Lab 1):** trích xuất mặt nạ làn  
- **Geometry (Lab 2):** ROI → tinh chỉnh → chiếu BEV  
- **Lane estimation (Lab 3.1):** multi-ratio sampling + adaptive selection  
- **Control (Lab 3.2):** vị trí ngang + heading → lệnh đánh lái  
- **Visualization:** hiển thị BEV, điểm lấy mẫu và thông tin steering  

Vòng lặp dưới đây thực hiện tất cả các bước trên theo từng frame:

1. Tải video từ đĩa  
2. Áp dụng segmentation và làm sạch ROI  
3. Chiếu mặt nạ sang BEV  
4. Tính toán hình học tại nhiều tỉ lệ  
5. Chọn tỉ lệ ổn định nhất  
6. Sinh lệnh đánh lái  
7. Vẽ overlay phục vụ debug  
8. Lưu các frame đã xử lý thành một video đầu ra


### Step 1 — Load Input Video

Trong bước này, chúng ta chỉ định đường dẫn đến video lái xe sẽ được xử lý bởi toàn bộ pipeline giữ làn.  
Cell này bao gồm:

- một `video_path` cho phép người dùng chỉnh sửa  
- kiểm tra tự động sự tồn tại của file  
- yêu cầu nhập lại đường dẫn nếu file không tồn tại  
- khởi tạo đối tượng đọc video của OpenCV  

Điều này đảm bảo pipeline luôn bắt đầu với một video đầu vào hợp lệ và cung cấp phản hồi rõ ràng cho người học.


In [ ]:
video_path = r"C:\Users\admin\ACE_images\esp32_capture.mp4"
# Optional fallback if file is missing
if not os.path.exists(video_path):
    print(f"[INFO] The path '{video_path}' does not exist.")
    print("Please enter a valid video file path:")
    video_path = input("Video path: ").strip()

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise FileNotFoundError(f"Could not open video: {video_path}")

print(f"[OK] Loaded input video: {video_path}")


### Step 2 — Configure Output Video Writer

Ở bước này, chúng ta cấu hình video writer để lưu các frame đã được pipeline xử lý.

Cell này thực hiện:
- lấy độ phân giải và FPS từ video đầu vào  
- tạo writer MP4 bằng OpenCV  
- hiển thị thông tin xác nhận về vị trí lưu và tham số video  

Các frame trực quan hóa sau xử lý sẽ được ghi vào `lab3_output.mp4`.


In [ ]:
frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_in       = cap.get(cv2.CAP_PROP_FPS) or 30

output_path = "lab3_output.mp4"

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(
    output_path,
    fourcc,
    fps_in,
    (frame_width, frame_height)
)

print(f"[OK] Output will be saved to: {output_path}")
print(f"[INFO] Resolution: {frame_width}x{frame_height}, FPS: {fps_in}")


### Step 3 — Run the Full Lane-Keeping Pipeline

Cell này tích hợp toàn bộ các thành phần đã được xây dựng trong các lab trước và chạy chúng theo từng frame trên video đầu vào.

Pipeline bao gồm:

1. ***Segmentation (Lab 1)***  
   Sinh mặt nạ làn từ mỗi frame đầu vào và tinh chỉnh bằng ROI + morphology.

2. ***BEV Projection (Lab 2)***  
   Chuyển mặt nạ làn sang biểu diễn bird’s-eye-view để trích xuất hình học dễ dàng hơn.

3. ***Multi-Ratio Geometry Sampling (Lab 3.1)***  
   Ước lượng tâm làn và heading tại nhiều tỉ lệ lấy mẫu theo chiều dọc.

4. ***Adaptive Ratio Selection (Lab 3.1)***  
   Tự động chọn hình học ổn định nhất trong các tỉ lệ đã lấy mẫu.

5. ***Controller Computation (Lab 3.2)***  
   Chuyển hình học làn thành lệnh đánh lái và nhãn hướng lái.

6. ***Visualization Overlay***  
   Vẽ BEV, các điểm tỉ lệ và thông tin debug steering lên frame.

7. ***Video Export***  
   Ghi frame đã xử lý vào video đầu ra.

Khi vòng lặp kết thúc, video đã xử lý cuối cùng được lưu thành `lab3_output.mp4`.


In [ ]:
# ======================================================
# Lane-Keeping Full Pipeline (End-to-End)
# ======================================================

proj = BEVProjector()
ratios = [0.98, 0.92, 0.82, 0.72]
ROI = np.float32([[0.03,0.58],[0.97,0.58],[0.97,0.99],[0.03,0.99]])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # --------------------------------------------------
    # (1) Segmentation + ROI refinement
    # --------------------------------------------------
    mask = backend.infer_mask01(frame)
    mask = apply_roi(mask, ROI)
    mask = refine_mask01(mask)

    # --------------------------------------------------
    # (2) BEV projection
    # --------------------------------------------------
    bev = proj.warp(mask)
    bev01 = bev   # alias

    # --------------------------------------------------
    # (3) Multi-ratio geometry sampling
    # --------------------------------------------------
    centers, pos_list, head_list = multi_ratio_measure(bev, ratios)

    # --------------------------------------------------
    # (4) Adaptive ratio selection
    # --------------------------------------------------
    sel = adaptive_select(centers, pos_list, head_list, ratios)
    pos    = sel["pos"]
    head   = sel["head"]
    r_star = sel["best_ratio"]

    # --------------------------------------------------
    # (5) Steering computation
    # --------------------------------------------------
    steer = controller(pos, head)
    label = steering_label(steer)

    # --------------------------------------------------
    # (6) Ratio → pixel conversion for visualization
    # --------------------------------------------------
    H = bev.shape[0]
    cy = int(r_star * H)
    cx = center_x(bev, r_star)

    # --------------------------------------------------
    # (7) Overlay visualization
    # --------------------------------------------------
    overlay = draw_overlay(
        frame_bgr   = frame,
        bev01       = bev,
        M_inv       = proj.M_inv,
        ratios      = ratios,
        centers_px  = centers,
        primary_idx = ratios.index(r_star),
        pos_m       = pos,
        head_deg    = head,
        dir_label   = label
    )

    out.write(overlay)

cap.release()
out.release()
print("DONE — saved to lab3_output.mp4")

### Hoàn thành Lab

Bạn đã hoàn thành bài lab cuối cùng của học phần.  
Tại thời điểm này, toàn bộ các thành phần cốt lõi đã được tích hợp thành một hệ thống giữ làn hoàn chỉnh, xuyên suốt từ thu nhận hình ảnh thời gian thực, phân đoạn làn bằng học sâu, xử lý hình học, ổn định tín hiệu cho đến điều khiển vòng kín.

Lab này tổng hợp và củng cố các kiến thức, kỹ năng đã học trong suốt khóa học, cho thấy cách các khái niệm lý thuyết được kết nối để tạo nên một hệ thống tự hành hoạt động thực tế. Pipeline hoàn chỉnh này là nền tảng vững chắc cho các thí nghiệm nâng cao, tối ưu hiệu năng và các hướng nghiên cứu tiếp theo trong lĩnh vực xe tự hành và hệ thống thông minh.